In [1]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("kaggle_eegs").getOrCreate()


24/01/28 15:17:11 WARN Utils: Your hostname, debianlab resolves to a loopback address: 127.0.1.1; using 192.168.1.155 instead (on interface enp2s0)
24/01/28 15:17:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/28 15:17:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
file_path = '/home/harraz/my_tensorflow/venv/kaggle/eegs/kaggle/eegs_train/train.csv'

# Read labels CSV file into a DataFrame
train_y = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the DataFrame
train_y.show(10)


+----------+----------+------------------------+--------------+------------------+--------------------------------+----------+----------+----------------+------------+--------+--------+---------+---------+----------+
|    eeg_id|eeg_sub_id|eeg_label_offset_seconds|spectrogram_id|spectrogram_sub_id|spectrogram_label_offset_seconds|  label_id|patient_id|expert_consensus|seizure_vote|lpd_vote|gpd_vote|lrda_vote|grda_vote|other_vote|
+----------+----------+------------------------+--------------+------------------+--------------------------------+----------+----------+----------------+------------+--------+--------+---------+---------+----------+
|1628180742|         0|                     0.0|        353733|                 0|                             0.0| 127492639|     42516|         Seizure|           3|       0|       0|        0|        0|         0|
|1628180742|         1|                     6.0|        353733|                 1|                             6.0|3887563113|     4

In [3]:
file_path = '/home/harraz/my_tensorflow/venv/kaggle/eegs/kaggle/eegs_train/'

# Read CSV file into a DataFrame
train_X = spark.read.parquet(file_path, inferSchema=False)

# Show the DataFrame
train_X.show(10)

+------+------+------+------+------+------+------+------+-----+-----+-----+------+------+------+------+------+------+-----+------+-----+
|   Fp1|    F3|    C3|    P3|    F7|    T3|    T5|    O1|   Fz|   Cz|   Pz|   Fp2|    F4|    C4|    P4|    F8|    T4|   T6|    O2|  EKG|
+------+------+------+------+------+------+------+------+-----+-----+-----+------+------+------+------+------+------+-----+------+-----+
|-42.11|-29.85|-11.75|   0.9|-46.79|-58.51|-46.76| -3.15| 1.94|44.35|24.83|-33.15|-61.54|-16.39| 16.18|-56.23|-37.86| -6.5| -6.31|-5.63|
|-37.88|-26.68|-12.61| -1.25|-43.48|-58.28|-49.03| -5.92| 3.91|44.44|21.42|-28.72|-55.35|-15.21| 14.24| -49.8| -39.5| -7.0| -6.44|-1.36|
|-44.89|-28.87| -13.7| -4.53|-44.32|-60.15|-51.88|-10.11|  1.0|45.67|18.26|-33.62|-51.27|-15.53| 12.69|-46.76|-33.41|-7.38| -7.63| 1.18|
|-30.95|-21.98|-10.18| -4.15| -38.4|-57.27|-50.53|-11.32| 7.41|48.78|18.54|-19.05|-40.24|-12.39| 13.28|-37.09|-27.82|-8.45|  -8.8|-4.78|
|-16.65|-11.68| -4.09| -0.36|-30.12|-50.9

In [5]:
from pyspark.sql.functions import when
train_Labels = train_y.withColumn('expert_consensus', when(train_y['expert_consensus'] == 'Seizure', 1).otherwise(0))
train_Labels.select(train_Labels.expert_consensus).show(10)


+----------------+
|expert_consensus|
+----------------+
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               1|
|               0|
+----------------+
only showing top 10 rows

